In [1]:
!pip -qq install langchain
!pip -qq install langchain-core
!pip -qq install langchain-community

In [6]:
!pip install ollama langchain beautifulsoup4 chromadb gradio -q

In [7]:
from langchain_community.llms import Ollama
llm = Ollama(model = "llama3.1")
llm.invoke("what is the Meaning of life")

"The question of the meaning of life has puzzled philosophers, theologians, scientists, and thinkers for centuries. While there's no one definitive answer, here are some perspectives on this profound topic:\n\n**Philosophical perspectives:**\n\n1. **Hedonism**: Life's purpose is to seek pleasure and avoid pain.\n2. **Utilitarianism**: The meaning of life is to maximize happiness and minimize suffering for the greatest number of people.\n3. **Existentialism**: Life has no inherent meaning; individuals must create their own purpose and values.\n\n**Spiritual and religious perspectives:**\n\n1. **Christianity**: The purpose of life is to love God and follow His commandments, with eternal salvation being the ultimate goal.\n2. **Buddhism**: The meaning of life is to attain enlightenment (Nirvana) through meditation, self-awareness, and letting go of attachment to worldly desires.\n3. **Islam**: Life's purpose is to worship Allah and live according to His will, as outlined in the Quran.\n\n

In [12]:
import ollama
ollama.pull(model="nomic-embed-text")
ollama.pull(model="llama3.1")

{'status': 'success'}

In [30]:
import gradio as gr
import ollama
from bs4 import BeautifulSoup as bs
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS, Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain.docstore.document import Document
import PyPDF2
import requests
import torch

# Load the data from the PDF
pdf_path = 'https://arxiv.org/pdf/2404.11597'

def load_pdf(pdf_path):
    response = requests.get(pdf_path)
    with open("document.pdf", "wb") as f:
        f.write(response.content)
        
    text = ""
    with open("document.pdf", "rb") as f:
        reader = PyPDF2.PdfReader(f)
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text += page.extract_text()
    return text

pdf_text = load_pdf(pdf_path)
docs = [Document(page_content=pdf_text, metadata={})]

# Split the loaded documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Create Ollama embeddings and vector store, ensuring CUDA usage
embeddings = OllamaEmbeddings(model="nomic-embed-text")

# Ensure that the embeddings use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

# Define the function to call the Ollama Llama3 model
def ollama_llm(question, context):
    system_message = (
        "You are a helpful assistant. Answer the question based on the given context. "
        "If the answer is not in the provided text, just respond with 'I do not know'."
    )
    formatted_prompt = f"Question: {question}\n\nContext: {context}"
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': formatted_prompt}
    ]
    response = ollama.chat(model='llama3.1', messages=messages)
    return response['message']['content']

# Define the RAG setup
retriever = vectorstore.as_retriever()

def rag_chain(question):
    retrieved_docs = retriever.invoke(question)
    formatted_context = "\n\n".join(doc.page_content for doc in retrieved_docs)
    answer = ollama_llm(question, formatted_context)
    return f"Answer: {answer}\n\nUsed Text for Inference:\n{formatted_context}"

# Define the Gradio interface
def get_important_facts(question):
    return rag_chain(question)

# Create a Gradio app interface
iface = gr.Interface(
  fn=get_important_facts,
  inputs=gr.Textbox(lines=2, placeholder="Enter your question here..."),
  outputs="text",
  title="RAG with Llama3.1",
  description="Ask questions about the provided context",
)

# Launch the Gradio app
iface.launch()


Using device: cuda
Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
